In [1]:
import warnings
warnings.filterwarnings('ignore')


# Все import соберем в начале проекта:
import pandas as pd
import re
from fuzzywuzzy import fuzz, process


# Введем глобальные переменные:
MAX_LENGTH_WORD = 5 # Количество выводимых найденных названий
LEVENSHTEIN_DISTANCE_MAX = 50 


def clean_names_goods(product: str) -> str:
    '''
    Функция для очистки названий товаров выполняет следующие действия:
    - Перевод названии товаров в нижний регистр.
    - Удаление символа 'запятая' в именах товаров.
    - Удаление из имени товара повторяющихся пробелов.
    - Удаление из имени товара ненужных, одиноких символов.
    - Удаление пробелов в начале и в конце названия товара.
    '''   
    # Переведём все названия товаров в нижний регистр:
    product = str(product).lower()
    # Удалим лишние символы 'запятая', 'слэш' ...:
    product = re.sub(r'[^\:\w\s]', ' ', product)
    # Убираем из имени товара повторяющиеся пробелы:
    product = re.sub(r'\s+', ' ', product)
    # Приведём название Производителя в написании на английском:
    product = product.replace('просепт', 'prosept')
    # Уберём пробелы в начале и в конце информационных признаков:
    product = re.sub(r'^ +| +$', ' ', product)
    return product


# Для решения задачи извлечения слов из сплошь записанного выражения
# не будем загружать огромные многомегабайтные библиотеки слов.
# В связи с ограниченным набором слов в названиях товаров, создадим свои,
# небольшие библиотеки, работающие практически мгновенно.


list_product_word = [
    'prosept', 'концентрат', 'crystal', 'готовый', 'duty', 'multipower', 
    'cooky', 'diona', 'готовое', 'ultra', 'antifoam', 'bath', 'universal', 
    'carpet', 'концентрированное', 'flox', 'эффектом', 'splash', 'epoxy', 
    'candy', 'optic', 'clean', 'шампунь', 'штуки', 'невымываемый'
    ]


def clean_name_products(product: str) -> str:
    '''
    Функция разделения предложения, написанных слитно, без пробелов.
    '''
    result = product
    for word in list_product_word:
        tmp_str = result.split(str(word))
        if len(tmp_str) > 1:
            result = tmp_str[0] + ' ' + word + ' ' + tmp_str[1]
    return result


def preparation_product_df(row: str) -> str:
    '''
    Функция полной обработки датафрейма с 
    названиями товара производителя.    
    '''
    return preparation_product_name(row)
    

def preparation_product_name(product: str) -> str:
    '''
    Функция обработки и подготовки к проведению поиска
    строки из названия товара производителя.
    '''
    # Проведём очистку названий товаров
    product = clean_names_goods(product)
    # Разделим предложения, написанных слитно на слова
    product = clean_name_products(product)
    # Проведём очистку названий товаров
    product = clean_names_goods(product)
    return product


def get_suitable_products(dealer_product: str) -> list:
    '''
    Функция поиска наименований товара.
    '''
    suitable_products = []
    for product in manufacturer_data['name']:
        l_d = fuzz.token_sort_ratio(dealer_product, product)
        if (l_d >= LEVENSHTEIN_DISTANCE_MAX):
            manufactur_product_id = manufacturer_data[
                manufacturer_data['name'] == product
            ]['id'].to_string(index=False)
            suitable_products.append({
                'id':  manufactur_product_id, 
                'product_name': product, 
                'levenshtein_distance': l_d 
            })
    return suitable_products


def get_solution(dealer_product: str) -> list:
    '''
    Функция структурированного вывода результатов поиска
    наименований товара
    '''
    suitable_solution = get_suitable_products(dealer_product)
    solution = sorted(suitable_solution,
                      key=lambda x: x['levenshtein_distance'], 
                      reverse=True)
    last_index = (MAX_LENGTH_WORD - 1 
                  if MAX_LENGTH_WORD < len(solution) 
                  else len(solution) - 1)
    
    return solution[0: last_index]



# Загрузка файлов с данными по названиям товара производителя.
df_product = pd.read_csv('data/marketing_product.csv', sep = ';')

# Оставим в датафрейме всего 2 колонки:
manufacturer_data = df_product[['id','name']]
# Почистим названия товароы в датасете:
# manufacturer_data['name'] = manufacturer_data['name'].\
#                             apply(preparation_product_df)

In [2]:
name_good = 'антисептик неВымываемый prosept l ultra, концентрат 1:10 / 1 л'
get_solution(name_good)

[{'id': '249',
  'product_name': 'Антисептик невымываемыйPROSEPT ULTRAкоричневый концентрат 1:10 / 1 л',
  'levenshtein_distance': 78},
 {'id': '248',
  'product_name': 'Антисептик невымываемый PROSEPT ULTRAконцентрат  1:10  / 30 л',
  'levenshtein_distance': 78},
 {'id': '250',
  'product_name': 'Антисептик невымываемыйPROSEPT ULTRAкоричневый концентрат 1:10 / 5 л',
  'levenshtein_distance': 77},
 {'id': '252',
  'product_name': 'Антисептик невымываемыйPROSEPT ULTRAкоричневый концентрат 1:10 / 30л',
  'levenshtein_distance': 74}]

In [4]:
name_good = 'антисептик неВымываемый prosept l ultra, концентрат 1:10 / 1 л'
# Почистим названия товаров в датасете:
manufacturer_data['name'] = manufacturer_data['name'].\
                            apply(preparation_product_df)
# Загружаем в поиск почищенный запрос:
get_solution(preparation_product_name(name_good))

[{'id': '245',
  'product_name': 'антисептик невымываемый prosept ultra концентрат 1:10 1 л',
  'levenshtein_distance': 98},
 {'id': '246',
  'product_name': 'антисептик невымываемый prosept ultra концентрат 1:10 5 л',
  'levenshtein_distance': 97},
 {'id': '248',
  'product_name': 'антисептик невымываемый prosept ultra концентрат 1:10 30 л',
  'levenshtein_distance': 96},
 {'id': '247',
  'product_name': 'антисептик невымываемый prosept ultra концентрат 1:10 20 л',
  'levenshtein_distance': 96}]